# Set up Signals for real-time calculation

Welcome to the [Snowplow Signals](https://docs.snowplow.io/docs/signals/) Quick Start [tutorial](https://docs.snowplow.io/tutorials/signals-quickstart/start).

This notebook is intended to be used as part of the tutorial. It's hosted in Google Colab so you won't need to configure anything locally.

Check out the tutorial pages to follow along.


## Install and connect to Signals

In [ ]:
%pip install snowplow-signals

To connect to your deployment, initialize the Signals object with your API credentials:


In [ ]:
from snowplow_signals import Signals
from google.colab import userdata

signals = Signals(
    api_url=userdata.get('SP_API_URL'),
    api_key=userdata.get('SP_API_KEY'),
    api_key_id=userdata.get('SP_API_KEY_ID'),
    org_id=userdata.get('SP_ORG_ID'),
)

## Define what attributes to calculate
### Define attributes

In [ ]:
from snowplow_signals import Attribute, Event, Criteria, Criterion, AtomicProperty
from datetime import timedelta

page_view_count = Attribute(
    name="page_view_count",
    description="Page views in the last 5 minutes.",
    type="int32",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="counter",
    period=timedelta(minutes=5),
)

most_recent_browser = Attribute(
    name="most_recent_browser",
    description="The last browser name tracked.",
    type="string",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="last",
    property="contexts_nl_basjes_yauaa_context_1[0].agentName",
)

first_referrer = Attribute(
    name="first_referrer",
    description="The first referrer tracked.",
    type="string",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="first",
    property="refr_urlhost",
    criteria=Criteria(
        all=[
            Criterion.neq(
                AtomicProperty(name="page_referrer"),
                ""
            )
        ]
    ),
    default_value=None
)

### Define an attribute group
Update the code to use your own email address before running.

In [ ]:
from snowplow_signals import StreamAttributeGroup, domain_sessionid

my_attribute_group = StreamAttributeGroup(
    name="my_quickstart_attribute_group",
    version=1,
    attribute_key=domain_sessionid,
    owner="user@company.com", # UPDATE THIS
    attributes=[
        page_view_count,
        most_recent_browser,
        first_referrer
    ],
)

### Test the definitions

In [ ]:
data = signals.test(
    attribute_group=my_attribute_group,
)
data

### Define a service
Update the code to use your own email address before running.

In [ ]:
from snowplow_signals import Service

my_service = Service(
    name='my_quickstart_service',
    owner="user@company.com", # UPDATE THIS
    attribute_groups=[my_attribute_group]
)

## Deploy configuration

In [ ]:
sp_signals.publish([my_attribute_group, my_service])

The configuration is now live.

## Retrieve calculated attributes

Follow the tutorial instructions to find your current session ID from outgoing web events. Provide it as `identifier` below, to retrieve attributes that Signals has just calculated about your session.

In [ ]:
response = sp_signals.get_service_attributes(
    source="my_quickstart_service",
    attribute_key="domain_sessionid",
    identifier="d99f6db1-7b28-46ca-a3ef-f0aace99ed86", # UPDATE THIS
)

df=response.to_dataframe()
df